In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import autogluon
from autogluon.tabular import TabularDataset,TabularPredictor
from sklearn.model_selection import train_test_split
import utils
import geopandas as gpd

In [2]:
gdf_json = utils.merge_geojson_files(r'C:\Users\km20186\project\signature\data\train\浸水想定区域\10_計画規模')
display(gdf_json.head())

305個のgeojsonファイルを全CPUコアを使用して並列読み込みします...
結合完了: 合計 7172706 件のフィーチャー


,A31a_101,A31a_102,A31a_103,A31a_104,A31a_105,A31a_106,A31a_107,A31a_108,geometry
0,8101010001,天塩川,81,北海道開発局,1,None,None,None,"POLYGON ((142.36388 44.26004, 142.36388 44.260..."
1,8101010001,天塩川,81,北海道開発局,1,None,None,None,"POLYGON ((142.38844 44.25612, 142.38844 44.256..."
2,8101010001,天塩川,81,北海道開発局,1,None,None,None,"POLYGON ((142.38719 44.25354, 142.38719 44.253..."
3,8101010001,天塩川,81,北海道開発局,1,None,None,None,"POLYGON ((142.05044 44.83904, 142.05044 44.839..."
4,8101010001,天塩川,81,北海道開発局,1,None,None,None,"POLYGON ((142.06338 44.8405, 142.06338 44.8405..."


In [3]:
gdf_json.shape

(7172706, 9)

In [11]:
# pandasの表示設定
pd.set_option('display.max_columns', None)  # すべての列を表示
pd.set_option('display.max_rows', None)     # すべての行を表示（オプション）
#pd.set_option('display.width', None)        # 行の折り返しを防ぐ
pd.set_option('display.max_colwidth', None) # 列の内容を省略しない

In [ ]:
drop_cols=[
    "nl","el",'building_name_ruby', 'name_ruby', 'school_ele_code', 'school_jun_code', 'money_hoshou_company', 'free_rent_duration', 'free_rent_gen_timing'
    ]
land_price_cols=[
    "L02_001",#用途区分
    "L02_006",#調査価格
    "L02_024",#地積
    "L02_029",#水道
    "L02_037",#道路区分
    "L02_045",#最寄り駅までの道路距離
    "L02_055",
    "L02_056",
    "L02_057",
    "L02_058",
    "L02_059",
    "L02_060",
    "L02_061",
    "L02_062",
    "L02_063",
    "L02_064",
    "L02_065",
    "L02_066",
    "L02_067",
    "L02_068",
    "L02_069",
    "L02_070",
    "L02_071",
    "L02_072",
    "L02_073",
    "L02_074",
    "L02_075",
    "L02_076",
    "L02_077",
    "L02_078",
    "L02_079",
    "L02_080",
    "L02_081",
    "L02_082",
    "L02_083",
    "L02_084",
    "L02_085",
    "L02_086",
    "L02_087",
    "L02_088",
    "L02_089",
    "L02_090",
    "L02_091",
    "L02_092",
    "L02_093",
    "L02_094",
    "geometry"
    ]

In [15]:
gdf_json = gpd.read_file(os.path.join('..', 'data', 'train', 'L02-23.geojson'))#[land_price_cols]
gdf_json2 = gpd.read_file(os.path.join('..', 'data', 'train', 'L02-24.geojson'))#[land_price_cols]


In [16]:
gdf_json.crs

<Geographic 2D CRS: EPSG:6668>
Name: JGD2011
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: Japan - onshore and offshore.
- bounds: (122.38, 17.09, 157.65, 46.05)
Datum: Japanese Geodetic Datum 2011
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [3]:
train_df = pd.read_csv(os.path.join('..', 'data', 'train', 'train.csv'))
test_df = pd.read_csv(os.path.join('..', 'data', 'test', 'test.csv'))

C:\Users\km20186\AppData\Local\Temp\ipykernel_21812\3353662060.py:1: DtypeWarning: Columns (63) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(os.path.join('..', 'data', 'train', 'train.csv'))
C:\Users\km20186\AppData\Local\Temp\ipykernel_21812\3353662060.py:2: DtypeWarning: Columns (46,55,56,63,146) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(os.path.join('..', 'data', 'test', 'test.csv'))


In [ ]:
train_gdf=utils.convert_to_geodataframe(train_df.drop_duplicates(subset=['lon','lat']))
train_gdf.head()

,target_ym,money_room,lon,lat
2,201901,24480000,136.644708,35.072248
4,201901,18800000,136.875754,35.002403
6,201901,9900000,136.872669,35.000171
13,201901,16000000,136.871552,34.989686
14,201901,16800000,136.907383,34.964279
16,201901,16700000,136.911099,34.963407
17,201901,14480000,136.911099,34.963407
18,201901,32000000,136.850106,34.979704
19,201901,16000000,136.868058,34.976485
21,201901,18000000,136.282109,35.311584


In [5]:
train_gdf=utils.convert_to_geodataframe(train_df)
train_gdf=utils.nearest_merge(train_gdf,gdf_json,threshold=1500)
train_gdf=utils.nearest_merge(train_gdf,gdf_json2,threshold=1500)
#display(train_gdf.head())


In [ ]:
test_df = pd.read_csv(os.path.join('..', 'data', 'test', 'test.csv'))
test_gdf=utils.convert_to_geodataframe(test_df)
test_gdf=utils.nearest_merge(test_gdf,gdf_json,threshold=1500)
test_gdf["land_price_x"]=test_gdf.geometry.x
test_gdf["land_price_y"]=test_gdf.geometry.y
test_df = pd.DataFrame(test_gdf.drop(columns=[col for col in test_gdf.columns if 'geometry' in col]))

C:\Users\km20186\AppData\Local\Temp\ipykernel_32892\1463719098.py:1: DtypeWarning: Columns (46,55,56,63,146) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(os.path.join('..', 'data', 'test', 'test.csv'))


In [15]:
# distance列がNULLとなっている割合を計算
null_ratio = train_gdf['distance'].isnull().sum() / len(train_gdf)
print(f"distance列のNULL割合: {null_ratio:.2%} ({train_gdf['distance'].isnull().sum()}/{len(train_gdf)})")


distance列のNULL割合: 12.05% (43843/363924)


In [7]:
# GeoDataFrameをDataFrameに変換(geometry列を削除)
train_gdf["land_price_x"]=train_gdf.geometry.x
train_gdf["land_price_y"]=train_gdf.geometry.y
train_df = pd.DataFrame(train_gdf.drop(columns=[col for col in train_gdf.columns if 'geometry' in col]))

In [10]:
train, test = train_test_split(train_df, test_size=0.2, random_state=42)
train_data: pd.DataFrame = TabularDataset(train)
test_data: pd.DataFrame = TabularDataset(test)
model=TabularPredictor(
    label='money_room',
    problem_type='regression',
    eval_metric='mape'
)
predictor=model.fit(
    train_data=train_data,
    test_data=test_data,
    presets='medium_quality',
    time_limit=720,
)

No path specified. Models will be saved in: "AutogluonModels\ag-20251215_091731"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.10.19
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          20
Memory Avail:       11.33 GB / 31.66 GB (35.8%)
Disk Space Avail:   71.72 GB / 237.73 GB (30.2%)
Presets specified: ['medium_quality']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 720s
AutoGluon will save models to "c:\Users\km20186\project\signature\my_code\AutogluonModels\ag-20251215_091731"
Train Data Rows:    291139
Train Data Columns: 423
Label Column:       money_room
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    15242.33 MB
	Train Data (Original)  Memory Usage: 453

[1000]	valid_set's l2: 2.82512e+13	valid_set's mean_absolute_percentage_error: -0.167405
[2000]	valid_set's l2: 2.56862e+13	valid_set's mean_absolute_percentage_error: -0.159366
[3000]	valid_set's l2: 2.43949e+13	valid_set's mean_absolute_percentage_error: -0.154838
[4000]	valid_set's l2: 2.36331e+13	valid_set's mean_absolute_percentage_error: -0.152218
[5000]	valid_set's l2: 2.30413e+13	valid_set's mean_absolute_percentage_error: -0.150014
[6000]	valid_set's l2: 2.26075e+13	valid_set's mean_absolute_percentage_error: -0.148272
[7000]	valid_set's l2: 2.22665e+13	valid_set's mean_absolute_percentage_error: -0.146579
[8000]	valid_set's l2: 2.19808e+13	valid_set's mean_absolute_percentage_error: -0.145172
[9000]	valid_set's l2: 2.17262e+13	valid_set's mean_absolute_percentage_error: -0.143904


	Ran out of time, early stopping on iteration 9625. Best iteration is:
	[9617]	valid_set's l2: 2.16162e+13	valid_set's mean_absolute_percentage_error: -0.143271
	-0.1433	 = Validation score   (-mean_absolute_percentage_error)
	592.88s	 = Training   runtime
	0.65s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.00s of the -3.14s of remaining time.
	Ensemble Weights: {'LightGBMXT': 1.0}
	-0.1433	 = Validation score   (-mean_absolute_percentage_error)
	0.01s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 724.46s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 4495.6 rows/s (2912 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("c:\Users\km20186\project\signature\my_code\AutogluonModels\ag-20251215_091731")


In [11]:
predictor.leaderboard(test)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,-0.146985,-0.143271,mean_absolute_percentage_error,15.98495,0.646749,592.879773,15.98495,0.646749,592.879773,1,True,1
1,WeightedEnsemble_L2,-0.146985,-0.143271,mean_absolute_percentage_error,16.06017,0.647752,592.891341,0.07522,0.001003,0.011568,2,True,2


In [54]:
predictor.leaderboard(test)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,-0.151821,-0.14987,mean_absolute_percentage_error,8.931070,0.428252,268.771355,8.931070,0.428252,268.771355,1,True,1
1,WeightedEnsemble_L2,-0.151821,-0.14987,mean_absolute_percentage_error,8.985064,0.429255,268.780988,0.053993,0.001003,0.009634,2,True,2


In [47]:
test_data: pd.DataFrame = TabularDataset(test_df)
predictions = predictor.predict(test_data)
#

In [48]:
pred=pd.DataFrame(predictions)
pred.head()
pred.to_csv(os.path.join('..', 'data', 'submit', 'submission.csv'), index_label='id')

In [ ]:
train_data.head()

In [12]:
importance_df = predictor.feature_importance(test,num_shuffle_sets=1)

These features in provided data are not utilized by the predictor and will be ignored: ['building_name_ruby', 'name_ruby', 'school_ele_code', 'school_jun_code', 'money_hoshou_company', 'free_rent_duration', 'free_rent_gen_timing', 'L02_095_left', 'L02_017_right', 'L02_096_right']
Computing feature importance via permutation shuffling for 413 features using 5000 rows with 1 shuffle sets...
	990.61s	= Expected runtime (990.61s per shuffle set)
	1043.25s	= Actual runtime (Completed 1 of 1 shuffle sets)


In [16]:
display(importance_df)

,importance,stddev,p_value,n,p99_high,p99_low
house_area,2.648899e-01,NaN,NaN,1,NaN,NaN
year_built,1.863589e-01,NaN,NaN,1,NaN,NaN
L02_006_right,7.911921e-02,NaN,NaN,1,NaN,NaN
L02_006_left,6.232037e-02,NaN,NaN,1,NaN,NaN
L02_020_right,6.174244e-02,NaN,NaN,1,NaN,NaN
snapshot_land_area,3.928210e-02,NaN,NaN,1,NaN,NaN
L02_020_left,3.551897e-02,NaN,NaN,1,NaN,NaN
rosen_name1,3.122908e-02,NaN,NaN,1,NaN,NaN
L02_094_left,2.211417e-02,NaN,NaN,1,NaN,NaN
madori_kind_all,2.053067e-02,NaN,NaN,1,NaN,NaN
